In [2]:
# Importar as bibliotecas necessárias
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import joblib

# Carregar o dataset
df = pd.read_csv('/IMDB Dataset.csv')

# Mostrar as primeiras linhas para confirmar que carregou bem
print("Dados carregados com sucesso!")
print(df.head())

Dados carregados com sucesso!
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [3]:
# Verificar quantos positivos e negativos existem
print("\n--- Distribuição dos Dados ---")
print(df['sentiment'].value_counts())

# Verificar se há dados em falta
print("\n--- Dados em Falta ---")
print(df.isnull().sum())

# Ver um exemplo de crítica antes da limpeza (note as tags <br />)
print("\n--- Exemplo de Review Original (Sujo) ---")
print(df['review'][0])


--- Distribuição dos Dados ---
sentiment
positive    25000
negative    25000
Name: count, dtype: int64

--- Dados em Falta ---
review       0
sentiment    0
dtype: int64

--- Exemplo de Review Original (Sujo) ---
One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Musli

In [4]:
# Função para limpar o texto
def clean_text(text):
    # Remover tags HTML (como <br />)
    text = re.sub(r'<[^>]+>', '', text)
    # Remover caracteres especiais e números (manter apenas letras)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Converter para minúsculas
    text = text.lower()
    return text

# Aplicar a limpeza a todo o dataset (pode demorar alguns segundos)
print("A limpar os textos...")
df['cleaned_review'] = df['review'].apply(clean_text)

# Ver como ficou o exemplo depois da limpeza
print("\n--- Exemplo de Review Limpo ---")
print(df['cleaned_review'][0])

# Separar dados de Treino (80%) e Teste (20%)
X = df['cleaned_review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nDados divididos: {len(X_train)} treino, {len(X_test)} teste.")

A limpar os textos...

--- Exemplo de Review Limpo ---
one of the other reviewers has mentioned that after watching just  oz episode youll be hooked they are right as this is exactly what happened with methe first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the wordit is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to manyaryans muslims gangstas latinos christians italians irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awayi would say the main appeal of the show is due to the fact that it goe

In [7]:
# Criar o Pipeline (Vetorizador + Modelo)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')), # Remove palavras comuns como 'the', 'is'
    ('model', MultinomialNB()) # Algoritmo Naive Bayes
])

# Treinar o modelo
print("Treinando o modelo...")
pipeline.fit(X_train, y_train)
print("Modelo treinado!")

Treinando o modelo...
Modelo treinado!


In [8]:
# Fazer previsões nos dados de teste
y_pred = pipeline.predict(X_test)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)

print("--- Resultados da PoC ---")
print(f"Acurácia Final: {accuracy:.2%}") # Esperamos algo acima de 85%
print("\nRelatório Detalhado:")
print(classification_report(y_test, y_pred))

--- Resultados da PoC ---
Acurácia Final: 86.58%

Relatório Detalhado:
              precision    recall  f1-score   support

    negative       0.86      0.88      0.87      4961
    positive       0.88      0.86      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000

